In [ ]:
#https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/bigbigan_with_tf_hub.ipynb
import io
import IPython.display
import PIL.Image
from pprint import pformat
import numpy as np
import tensorflow as tf ##May report errors due to tensorflow versions
import tensorflow_hub as hub
import h5py
from datetime import datetime
from math import ceil
from random import shuffle
from sklearn import preprocessing
import os
import shutil
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
module_path = '/content/drive/My Drive/...Your BBG Model Path.../Bigbigan1'  # ResNet-50
os.chdir(module_path)
os.listdir(module_path)


with tf.Graph().as_default():

  def interleave(*args):
    """Interleaves input arrays of the same shape along the batch axis."""
    if not args:
      raise ValueError('At least one argument is required.')
    a0 = args[0]
    if any(a.shape != a0.shape for a in args):
      raise ValueError('All inputs must have the same shape.')
    if not a0.shape:
      raise ValueError('Inputs must have at least one axis.')
    out = np.transpose(args, [1, 0] + list(range(2, len(a0.shape) + 1)))
    out = out.reshape(-1, *a0.shape[1:])
    return out
  def imgrid(imarray, cols=4, pad=1, padval=255, row_major=True):
    """Lays out a [N, H, W, C] image array as a single image grid."""
    pad = int(pad)
    if pad < 0:
      raise ValueError('pad must be non-negative')
    cols = int(cols)
    assert cols >= 1
    N, H, W, C = imarray.shape
    rows = N // cols + int(N % cols != 0)
    batch_pad = rows * cols - N
    assert batch_pad >= 0
    post_pad = [batch_pad, pad, pad, 0]
    pad_arg = [[0, p] for p in post_pad]
    imarray = np.pad(imarray, pad_arg, 'constant', constant_values=padval)
    H += pad
    W += pad
    grid = (imarray
            .reshape(rows, cols, H, W, C)
            .transpose(0, 2, 1, 3, 4)
            .reshape(rows*H, cols*W, C))
    if pad:
      grid = grid[:-pad, :-pad]
    return grid
  def imshow(a, format='png', jpeg_fallback=True):
    """Displays an image in the given format."""
    a = a.astype(np.uint8)
    data = io.BytesIO()
    PIL.Image.fromarray(a).save(data, format)
    im_data = data.getvalue()
    try:
      disp = IPython.display.display(IPython.display.Image(im_data))
    except IOError:
      if jpeg_fallback and format != 'jpeg':
        print ('Warning: image was too large to display in format "{}"; '
              'trying jpeg instead.').format(format)
        return imshow(a, format='jpeg')
      else:
        raise
    return disp

  def image_to_uint8(x):
    """Converts [-1, 1] float array to [0, 255] uint8."""
    x = np.asarray(x)
    x = (256. / 2.) * (x + 1.)
    x = np.clip(x, 0, 255)
    x = x.astype(np.uint8)
    return x

  module = hub.Module(module_path, trainable = True) # inference
  #module = hub.load(module_path) # inference

  for signature in module.get_signature_names():
    print('Signature:', signature)
    print('Inputs:', pformat(module.get_input_info_dict(signature)))
    print('Outputs:', pformat(module.get_output_info_dict(signature)))
    print()

 

  #@tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

  def generate(z, upsample=False):
    """Run a batch of latents z through the generator to generate images.

    Args:
    z: A batch of 120D Gaussian latents, shape [N, 120].

    Returns: a batch of generated RGB images, shape [N, 128, 128, 3], range
    [-1, 1].
    """
    outputs = module(z, signature='generate', as_dict=True)
    return outputs['upsampled' if upsample else 'default']

  def make_generator_ph():
    """Creates a tf.placeholder with the dtype & shape of generator inputs."""
    info = module.get_input_info_dict('generate')['z']
    return tf.placeholder(dtype=info.dtype, shape=info.get_shape())

  def gen_pairs_for_disc(z):
    """Compute generator input pairs (G(z), z) for discriminator, given z.

    Args:
    z: A batch of latents (120D standard Gaussians), shape [N, 120].

    Returns: a tuple (G(z), z) of discriminator inputs.
    """
    # Downsample 256x256 image x for 128x128 discriminator input.
    x = generate(z)
    return x, z

  def encode(x, return_all_features=False):
    """Run a batch of images x through the encoder.

    Args:
    x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].
    return_all_features: If True, return all features computed by the encoder.
        Otherwise (default) just return a sample z_hat.

    Returns: the sample z_hat of shape [N, 120] (or a dict of all features if
    return_all_features).
    """
    outputs = module(x, signature='encode', as_dict=True)
    return outputs if return_all_features else outputs['z_sample']

  def make_encoder_ph():
    """Creates a tf.placeholder with the dtype & shape of encoder inputs."""
    info = module.get_input_info_dict('encode')['x']
    return tf.placeholder(dtype=info.dtype, shape=info.get_shape())

  def enc_pairs_for_disc(x):
    """Compute encoder input pairs (x, E(x)) for discriminator, given x.

    Args:
    x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].

    Returns: a tuple (downsample(x), E(x)) of discriminator inputs.
    """
    x_down = tf.nn.avg_pool(x, ksize=2, strides=2, padding='SAME')
    
    z = encode(x)
    return x_down, z

  def discriminate(x, z):
    """Compute the discriminator scores for pairs of data (x, z).

    (x, z) must be batches with the same leading batch dimension, and joint
    scores are computed on corresponding pairs x[i] and z[i].

    Args:
    x: A batch of data (128x128 RGB images), shape [N, 128, 128, 3], range
        [-1, 1].
    z: A batch of latents (120D standard Gaussians), shape [N, 120].

    Returns:
    A dict of scores:
        score_xz: the joint scores for the (x, z) pairs.
        score_x: the unary scores for x only.
        score_z: the unary scores for z only.
    """
    inputs = dict(x=x, z=z)
    return module(inputs, signature='discriminate', as_dict=True)

  def reconstruct_x(x, use_sample=True, upsample=False):
    """Compute BigBiGAN reconstructions of images x via G(E(x)).

    Args:
    x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].
    use_sample: takes a sample z_hat ~ E(x). Otherwise, deterministically
        use the mean. (Though a sample z_hat may be far from the mean z,
        typically the resulting recons G(z_hat) and G(z) are very
        similar.
    upsample: if set, upsample the reconstruction to the input resolution
        (256x256). Otherwise return the raw lower resolution generator output
        (128x128).

    Returns: a batch of recons G(E(x)), shape [N, 256, 256, 3] if
    `upsample`, otherwise [N, 128, 128, 3].
    """
    if use_sample:
        z = encode(x)
    else:
        z = encode(x, return_all_features=True)['z_mean']
    recons = generate(z, upsample=upsample)
    return recons

  def losses(x, z):
    """Compute per-module BigBiGAN losses given data & latent sample batches.

    Args:
    x: A batch of data (256x256 RGB images), shape [N, 256, 256, 3], range
        [-1, 1].
    z: A batch of latents (120D standard Gaussians), shape [M, 120].

    For the original BigBiGAN losses, pass batches of size N=M=2048, with z's
    sampled from a 120D standard Gaussian (e.g., np.random.randn(2048, 120)),
    and x's sampled from the ImageNet (ILSVRC2012) training set with the
    "ResNet-style" preprocessing from:

        https://github.com/tensorflow/tpu/blob/master/models/official/resnet/resnet_preprocessing.py

    Returns:
    A dict of per-module losses:
        disc: loss for the discriminator.
        enc: loss for the encoder.
        gen: loss for the generator.
    """
    # Compute discriminator scores on (x, E(x)) pairs.
    # Downsample 256x256 image x for 128x128 discriminator input.
    scores_enc_x_dict = discriminate(*enc_pairs_for_disc(x))
    scores_enc_x = tf.concat([scores_enc_x_dict['score_xz'],
                            scores_enc_x_dict['score_x'],
                            scores_enc_x_dict['score_z']], axis=0)

    # Compute discriminator scores on (G(z), z) pairs.
    scores_gen_z_dict = discriminate(*gen_pairs_for_disc(z))
    scores_gen_z = tf.concat([scores_gen_z_dict['score_xz'],
                            scores_gen_z_dict['score_x'],
                            scores_gen_z_dict['score_z']], axis=0)

    disc_loss_enc_x = tf.reduce_mean(tf.nn.relu(1. - scores_enc_x))
    disc_loss_gen_z = tf.reduce_mean(tf.nn.relu(1. + scores_gen_z))
    disc_loss = disc_loss_enc_x + disc_loss_gen_z

    enc_loss = tf.reduce_mean(scores_enc_x)
    gen_loss = tf.reduce_mean(-scores_gen_z)

    return disc_loss, enc_loss, gen_loss

  with tf.Session() as sess:

    lr = 0.0001
    batch_size = 20
    # Make input placeholders for x (`enc_ph`) and z (`gen_ph`).
    enc_ph = make_encoder_ph()
    gen_ph = make_generator_ph()

    # Compute samples G(z) from encoder input z (`gen_ph`).
    gen_samples = generate(gen_ph)

    # Compute reconstructions G(E(x)) of encoder input x (`enc_ph`).
    recon_x = reconstruct_x(enc_ph, upsample=True)

    # Compute encoder features used for representation learning evaluations given
    # encoder input x (`enc_ph`).
    enc_features = encode(enc_ph, return_all_features=True)

    # Compute discriminator scores for encoder pairs (x, E(x)) given x (`enc_ph`)
    # and generator pairs (G(z), z) given z (`gen_ph`).
    disc_scores_enc = discriminate(*enc_pairs_for_disc(enc_ph))
    disc_scores_gen = discriminate(*gen_pairs_for_disc(gen_ph))

    # Compute losses.
    loss_disc, loss_enc, loss_gen = losses(enc_ph, gen_ph)

    optimizer = tf.train.AdamOptimizer(lr, beta1 = 0.9)

    train_op_disc = optimizer.minimize(loss_disc)
    train_op_enc = optimizer.minimize(loss_enc)
    train_op_gen = optimizer.minimize(loss_gen)

    init = tf.global_variables_initializer()
    sess.run(init)

    time = '{}'.format(datetime.now().strftime('%b%d_%H%M')) #运行新模型时使用

    path = "Training Data Path"
    os.chdir(path) 
    os.listdir(path)

    for batch_index in range(200):
      
      with h5py.File("Training.hdf5", 'r') as record:
        keys = list(record.keys())
        data_num = len(record)
        batches_list = list(range(int(ceil(float(data_num) / batch_size))))
        shuffle(batches_list)
        #print(batches_list)
        for n, i in enumerate(batches_list):
          i_s = i * batch_size  # index of the first image in this batch
          i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
          Ori_img = []
          Number_batch = i_e - i_s
          Ori_img_name = keys[i_s:i_e]
          for is_p in range(i_s, i_e):
            image = np.array(record[keys[is_p]]["train"])
            if is_p == i_s:
              Ori_img = image
              Ori_img = np.expand_dims(Ori_img, axis = 0)
              #print(Ori_img.shape)
            else:
              image = np.expand_dims(image, axis = 0)
              Ori_img = np.append(Ori_img, image, axis = 0)
              #print(Ori_img.shape)

          print(Ori_img.shape, n, i, batch_index)
          if((n + 1) % 1000 == 0):
            feed_dict = {gen_ph: np.random.randn(batch_size, 120)}
            _out_samples = sess.run(gen_samples, feed_dict=feed_dict)   
            #print('samples shape:', _out_samples.shape)
            imshow(imgrid(image_to_uint8(_out_samples), cols=4))

            _out_recons = sess.run(recon_x, feed_dict={enc_ph: Ori_img})
            #print('reconstructions shape:', _out_recons.shape)
            inputs_and_recons = interleave(Ori_img, _out_recons)
            #print('inputs_and_recons shape:', inputs_and_recons.shape)
            imshow(imgrid(image_to_uint8(inputs_and_recons), cols=4))
          
          feed_dict = {enc_ph: Ori_img, gen_ph: np.random.randn(batch_size, 120)}
          _train_op_disc,_train_op_enc,_train_op_gen = sess.run(
            [train_op_disc, train_op_enc, train_op_gen], feed_dict=feed_dict)
          
          _out_scores_enc, _out_scores_gen, _loss_disc, _loss_enc, _loss_gen = sess.run(
            [disc_scores_enc, disc_scores_gen, loss_disc, loss_enc, loss_gen], feed_dict=feed_dict)
          #print('Encoder scores:', {k: v.mean() for k, v in _out_scores_enc.items()})
          #print('Generator scores:', {k: v.mean() for k, v in _out_scores_gen.items()})
          #print('Losses:', _loss_disc, _loss_enc, _loss_gen)
        
      if ((batch_index + 1) % 4 == 0 and lr > 1e-7):
        lr *= 0.5

      if ((batch_index + 1) % 30 == 0):
        path = "Save Model Path"
        export_module_dir = os.path.join(path, "Save Model Folder")
        os.chdir(export_module_dir) 
        shutil.rmtree(export_module_dir)
        os.mkdir(export_module_dir)
        module.export(export_module_dir, sess)
        
      path = "Validation Data Path"
      os.chdir(path) 
      os.listdir(path)
      with h5py.File("Validation.hdf5", 'r') as record:
        keys = list(record.keys())
        #print(keys)
        data_num = len(record)
        batches_list = list(range(int(ceil(float(data_num) / batch_size))))
        shuffle(batches_list)
        #print(batches_list) 
        for n, i in enumerate(batches_list):
          i_s = i * batch_size  # index of the first image in this batch
          i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
          Ori_img = []
          Number_batch = i_e - i_s
          Ori_img_name = keys[i_s:i_e]
          for is_p in range(i_s, i_e):
            image = np.array(record[keys[is_p]]["train"])
            if is_p == i_s:
              Ori_img = image
              Ori_img = np.expand_dims(Ori_img, axis = 0)
              #print(Ori_img.shape)
            else:
              image = np.expand_dims(image, axis = 0)
              Ori_img = np.append(Ori_img, image, axis = 0)
              #print(Ori_img.shape)
          _out_features = sess.run(enc_features, feed_dict={enc_ph: Ori_img})

      path = "Test Data Path"
      os.chdir(path) 
      os.listdir(path)
      with h5py.File("Test.hdf5", 'r') as record:
        keys = list(record.keys())
        #print(keys)
        data_num = len(record)
        batches_list = list(range(int(ceil(float(data_num) / batch_size))))
        shuffle(batches_list)
        #print(batches_list) 
        for n, i in enumerate(batches_list):
          i_s = i * batch_size  # index of the first image in this batch
          i_e = min([(i + 1) * batch_size, data_num])  # index of the last image in this batch
          Ori_img = []
          Number_batch = i_e - i_s
          Ori_img_name = keys[i_s:i_e]
          for is_p in range(i_s, i_e):
            image = np.array(record[keys[is_p]]["train"])
            if is_p == i_s:
              Ori_img = image
              Ori_img = np.expand_dims(Ori_img, axis = 0)
              #print(Ori_img.shape)
            else:
              image = np.expand_dims(image, axis = 0)
              Ori_img = np.append(Ori_img, image, axis = 0)
              #print(Ori_img.shape)
          _out_features = sess.run(enc_features, feed_dict={enc_ph: Ori_img})


流式输出内容被截断，只能显示最后 5000 行内容。
(20, 256, 256, 3) 35 110 10
(20, 256, 256, 3) 36 170 10
(20, 256, 256, 3) 37 166 10
(20, 256, 256, 3) 38 145 10
(20, 256, 256, 3) 39 106 10
(20, 256, 256, 3) 40 123 10
(20, 256, 256, 3) 41 94 10
(20, 256, 256, 3) 42 64 10
(20, 256, 256, 3) 43 157 10
(20, 256, 256, 3) 44 151 10
(20, 256, 256, 3) 45 173 10
(20, 256, 256, 3) 46 131 10
(20, 256, 256, 3) 47 24 10
(20, 256, 256, 3) 48 233 10
(20, 256, 256, 3) 49 137 10
(20, 256, 256, 3) 50 61 10
(20, 256, 256, 3) 51 124 10
(20, 256, 256, 3) 52 208 10
(20, 256, 256, 3) 53 119 10
(20, 256, 256, 3) 54 142 10
(20, 256, 256, 3) 55 41 10
(20, 256, 256, 3) 56 17 10
(20, 256, 256, 3) 57 113 10
(20, 256, 256, 3) 58 47 10
(20, 256, 256, 3) 59 146 10
(20, 256, 256, 3) 60 138 10
(20, 256, 256, 3) 61 143 10
(20, 256, 256, 3) 62 39 10
(20, 256, 256, 3) 63 100 10
(20, 256, 256, 3) 64 218 10
(20, 256, 256, 3) 65 103 10
(20, 256, 256, 3) 66 8 10
(20, 256, 256, 3) 67 10 10
(20, 256, 256, 3) 68 246 10
(20, 256, 256, 3) 69 101 10
(20,